In [113]:
f = open('input7.txt', 'r')
input_commands = f.read()
input_commands = list(map(int, input_commands.split(',')))
f.close()

In [119]:
from enum import Enum
class Mode(Enum):
    Position = 0
    Immediate = 1

def op1(memory, position, modes):
    x,y,z = memory[position + 1], memory[position + 2], memory[position + 3]
    x_val = memory[x] if modes[0] == Mode.Position else x
    y_val = memory[y] if modes[1] == Mode.Position else y
    memory[z] = x_val + y_val
    return position + 4

def op2(memory, position, modes):
    x,y,z = memory[position + 1], memory[position + 2], memory[position + 3]
    x_val = memory[x] if modes[0] == Mode.Position else x
    y_val = memory[y] if modes[1] == Mode.Position else y
    memory[z] = x_val * y_val
    return position + 4

def op3(memory, position, modes, inputfunc):
    address = memory[position + 1]
    memory[address] = int(inputfunc())
    return position + 2

def op4(memory, position, modes, outputfunc):
    address = memory[position + 1]
    outputfunc(memory[address])
    return position + 2

def op5(memory, position, modes):
    x,y = memory[position + 1], memory[position + 2]
    x = memory[x] if modes[0] == Mode.Position else x
    y = memory[y] if modes[1] == Mode.Position else y
    if x == 0:
        return position + 3
    return y

def op6(memory, position, modes):
    x,y = memory[position + 1], memory[position + 2]
    x = memory[x] if modes[0] == Mode.Position else x
    y = memory[y] if modes[1] == Mode.Position else y
    if x == 0:
        return y
    return position  + 3

def op7(memory, position, modes):
    x,y,z = memory[position + 1], memory[position + 2], memory[position + 3]
    x = memory[x] if modes[0] == Mode.Position else x
    y = memory[y] if modes[1] == Mode.Position else y
    if x < y:
        memory[z] = 1
    else:
        memory[z] = 0
    return position + 4

def op8(memory, position, modes):
    x,y,z = memory[position + 1], memory[position + 2], memory[position + 3]
    x = memory[x] if modes[0] == Mode.Position else x
    y = memory[y] if modes[1] == Mode.Position else y
    if x == y:
        memory[z] = 1
    else:
        memory[z] = 0
    return position + 4


def get_mode_and_opcode(value):
    modes = [Mode.Position,Mode.Position]
    opcode = value
    str_val = str(value).rjust(4, '0')
    modes = list(map(lambda v: Mode(int(v)), (str_val[0:2])))[::-1]
    opcode = int(str_val[2:])
    return (modes, opcode)
        
    
def intcode(memory, position, inputfunc, outputfunc):
    modes, opcode = get_mode_and_opcode(memory[position])
    if opcode == 1:
        return op1(memory, position, modes)
    elif opcode == 2:
        return op2(memory, position, modes)
    elif opcode == 3:
        return op3(memory, position, modes, inputfunc)
    elif opcode == 4:
        return op4(memory,position, modes, outputfunc)
    elif opcode == 5:
        return op5(memory,position, modes)
    elif opcode == 6:
        return op6(memory,position, modes)
    elif opcode == 7:
        return op7(memory,position, modes)
    elif opcode == 8:
        return op8(memory,position, modes)
    elif opcode == 99:
        return None


In [140]:
phase_setting_seq = [1,0,4,3,2]

memory = [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,
1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]
        
def run_amplifier(memory, phase_setting, input_value):
    position = 0
    first_time = True
    def i_func():
        nonlocal phase_setting
        nonlocal input_value
        nonlocal first_time
        if first_time:
            first_time = False
            return phase_setting
        else:
            first_time = True
            return input_value
    output = 0
    def o_func(value):
        nonlocal output
        output = value
    while position != None:
        position = intcode(memory, position, i_func, o_func)
    return output
def check_phase_setting_seq(seq, memory):
    input_value = 0
    for phase_setting in seq:
        input_value = run_amplifier(memory.copy(), phase_setting, input_value)
    return input_value

print(check_phase_setting_seq(phase_setting_seq, memory.copy()))


65210


In [144]:
import itertools
memory = input_commands.copy()
sequences = list(itertools.permutations(range(5)))
test = list(map(lambda seq : check_phase_setting_seq(seq, input_commands.copy()), list(sequences)))
test.sort()
#print(test)
print(max(test))

70597


In [230]:
class amplifier:
    def __init__(self, memory, phase_setting):
        self.memory = memory
        self.phase_setting = phase_setting
        self.position = 0
        self.input_func_state = True
        self.output = 0
        self.halted = False
        self.output_ready = False
    def i_func(self):
        if self.input_func_state:
            self.input_func_state = False
            return self.phase_setting
        else:
            return self.input_value
    def o_func(self, value):
        self.output = value
        self.output_ready = True

    def run(self, input_value):
        self.input_value = input_value
        while not self.halted:
            self.position = intcode(self.memory, self.position, self.i_func, self.o_func)
            if self.position == None:
                self.halted = True
            if self.output_ready:
                self.output_ready = False
                return

In [231]:
phase_setting_seq = [9,8,7,6,5]
memory = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
#phase_setting_seq = [1,0,4,3,2]
#memory = [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]
amplifiers = []
for phase_setting in phase_setting_seq:
    amplifiers.append(amplifier(memory.copy(), phase_setting))
input_value = 0
while all(not amp.halted for amp in amplifiers):
    for amp in amplifiers:
        amp.run(input_value)
        input_value = amp.output

print(input_value)


139629729


In [234]:
memory = input_commands.copy()
sequences = list(itertools.permutations(range(5, 10)))
value = 0
for seq in sequences:
    amplifiers = []
    for phase_setting in seq:
        amplifiers.append(amplifier(memory.copy(), phase_setting))
    input_value = 0
    while all(not amp.halted for amp in amplifiers):
        for amp in amplifiers:
            amp.run(input_value)
            input_value = amp.output
    value = max(value, input_value)
print(value)

30872528
